In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [25]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
import ssl
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Embedding 
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.metrics import Precision, Recall
from gensim.models import Word2Vec
from tensorflow.keras.layers import LSTM, SpatialDropout1D, Bidirectional, Dropout

import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec

import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os

In [26]:
#download's the stop word's
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/calebschaefer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
dataset = load_dataset("yelp_review_full")
dp_train = dataset['train'].to_pandas()
dp_test = dataset['test'].to_pandas()

#used so we don't have to use the entire training dataset
dp_sample = dp_train.sample(n=20000, random_state=42)
dp_sample

Found cached dataset parquet (/Users/calebschaefer/.cache/huggingface/datasets/parquet/yelp_review_full-9c7006f5a2e02666/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

,label,text
177288,0,"First of all i'm not a big fan of buffet, i tr..."
238756,1,Thanks Yelp. I was looking for the words to de...
604225,2,Service was so-so. They were receiving a deliv...
2838,2,Stamoolis Brothers is one of the Strip Distric...
586957,0,I want to give a 2 stars because the service s...
...,...,...
188178,4,"It's not beautiful in appearance, but it remin..."
574411,0,I went to Fusion Pizza to try out a new place....
575952,0,"I cancelled my membership in June, and had sig..."
237244,1,Yikes...their standards have dropped yet again...


In [28]:
#preprocess's the text data
#by removing stop words and leading spaces 
#this data pre-process was inspired by: https://www.kaggle.com/code/gcdatkin/gru-hotel-rating-prediction
def pre_process_data(X):
    stop_words = stopwords.words('english')
    X = re.sub(r'\d+', ' ', X)
    X = X.split()
    X = " ".join([word for word in X if word.lower().strip() not in stop_words])
    return X

In [29]:
reviews = dp_sample['text'].apply(pre_process_data)
reviews

177288    First i'm big fan buffet, tried got $ credit s...
238756    Thanks Yelp. looking words describe place Meh ...
604225    Service so-so. receiving delivery might it. Fo...
2838      Stamoolis Brothers one Strip District storefro...
586957    want give stars service staff friendly good we...
                                ...                        
188178    beautiful appearance, reminds younger, hanging...
574411    went Fusion Pizza try new place. love business...
575952    cancelled membership June, signed six months t...
237244    Yikes...their standards dropped yet years ago....
390980    terrace really makes place. pretty cool, walls...
Name: text, Length: 20000, dtype: object

In [30]:
#provides the option a toke/sequence embedding approach
#takes the modified reviews
#return the modified inputs and the amount of words's to be used in an embedding layer
def create_sequences(sentences, num_words):
    #takes the most used word's so we don't have the entire vocab
    tokenizer = Tokenizer(num_words = num_words)
    tokenizer.fit_on_texts(sentences)

    sequences = tokenizer.texts_to_sequences(sentences)
    #get the length of the largest sequnces
    max_length = np.max(list(map(lambda x: len(x), sequences)))
    #pad all the input's to be the same length of the max length
    print(max_length)
    inputs = pad_sequences(sequences, maxlen=max_length, padding = 'post')
    
    return inputs, num_words
    
def make_embeddings(sentences):
    sentence_split = [line.split() for line in sentences]
    
    word2vec_model = Word2Vec(sentences=sentence_split, vector_size=100, window=5, min_count=1, workers=4)   
    vocab = list(word2vec_model.wv.key_to_index.keys())
    print(len(vocab))
    # Convert tokens to embeddings
    embedding_matrix = np.zeros((len(vocab), word2vec_model.vector_size))
    for index, word in enumerate(vocab):
        embedding_matrix[index] = word2vec_model.wv[word]
        
    
    indexed_data = []
    for sentence in sentence_split:
        indexed_sentence = [word2vec_model.wv.key_to_index[word] for word in sentence]
        indexed_data.append(indexed_sentence)
    max_length = np.max(list(map(lambda x: len(x), indexed_data)))
    padded_data = pad_sequences(indexed_data, maxlen=max_length, padding='post')
    
    
    
    return embedding_matrix, vocab, word2vec_model, padded_data
    
    

In [31]:
inputs, num_words = create_sequences(reviews,50000)
#embedding_matrix, vocab, word2vec_model, inputs = make_embeddings(reviews)

547


In [35]:
inputs

array([[  37,   24,  124, ...,    0,    0,    0],
       [ 751,  336,  138, ...,    0,    0,    0],
       [  10,  262,  262, ...,    0,    0,    0],
       ...,
       [3643, 2443, 3929, ...,    0,    0,    0],
       [3664, 4365, 1713, ...,    0,    0,    0],
       [4065,   14,  381, ...,    0,    0,    0]], dtype=int32)

In [36]:
labels = np.array(dp_sample['label'])
#creat's a one-hot ecnoding so we can use precision and recall in the tensorflow model
labels

array([0, 1, 2, ..., 0, 1, 2])

In [37]:
X_train, X_test, y_train, y_test = train_test_split(inputs, labels, train_size=0.75, random_state=42)
#y_train = tf.one_hot(y_train, depth=5)
#y_train = tf.cast(y_train, dtype=tf.int32)
#y_test =  tf.one_hot(y_test, depth=5)
#y_test = tf.cast(y_test, dtype=tf.int32)

X_train

array([[   49,  5292,  2835, ...,     0,     0,     0],
       [    9,    32,   298, ...,     0,     0,     0],
       [  117,   702,   702, ...,     0,     0,     0],
       ...,
       [  173,   571,   498, ...,     0,     0,     0],
       [  689,   349,   138, ...,     0,     0,     0],
       [ 7008,  2632, 10236, ...,     0,     0,     0]], dtype=int32)

In [45]:
model = keras.Sequential()
model.add(Embedding(num_words, 300, input_length = X_train.shape[1]))
model.add(Bidirectional(LSTM(128, dropout=0.2)))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))
model.summary()
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [24]:
#Tensorlflow mlp was inspired from their website
# ttps://www.tensorflow.org/guide/core/mlp_core and #https://www.geeksforgeeks.org/multi-layer-perceptron-learning-in-tensorflow
#input_shape = tf.keras.Input(shape=(max_length,))
#We want to use an embedding so the neural network is better able to differniate words

"""
model = Sequential([
    #add's an embedding layer from the word 2 vector that we learned
    #tf.keras.layers.Embedding(input_dim=len(vocab), output_dim =word2vec_model.vector_size,
                               #embeddings_initializer= tf.keras.initializers.Constant(embedding_matrix), trainable=False),
    #tf.keras.layers.Embedding(input_dim=embed_matrix.shape[0], output_dim=embed_matrix.shape[1], embeddings_initializer= tf.keras.initializers.Constant(embed_matrix), trainable=False),
    #tf.keras.layers.GlobalAveragePooling1D(),
    #Embedding(input_dim = num_words, output_dim = 128),
    #tf.keras.layers.GlobalAveragePooling1D(),
    #Flatten(),
    #Dense(256, activation = 'relu'),
    Dense(256, activation = 'relu'),
    Dense(256, activation = 'relu'),
    Dense(256, activation = 'relu'),
    
    #output layer
    Dense(5, activation = 'softmax'),
    
    
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])
"""

In [46]:
model.fit(X_train, y_train, epochs = 20,
    batch_size=128)

Epoch 1/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 165s 1s/step - accuracy: 0.2665 - loss: 1.5310
Epoch 2/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.5638 - loss: 1.0134
Epoch 3/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 205s 2s/step - accuracy: 0.6961 - loss: 0.7456
Epoch 4/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 205s 2s/step - accuracy: 0.7757 - loss: 0.5935
Epoch 5/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 204s 2s/step - accuracy: 0.8558 - loss: 0.4108
Epoch 6/20
 45/118 ━━━━━━━━━━━━━━━━━━━━ 2:05 2s/step - accuracy: 0.8999 - loss: 0.3068

KeyboardInterrupt: 

In [ ]:
tf.saved_model.save(model, "/Users/calebschaefer/Documents/comp_5600/term_project/model4")

In [49]:
model.evaluate(X_test, y_test)
predictions = model.predict(X_test)

157/157 ━━━━━━━━━━━━━━━━━━━━ 29s 187ms/step - accuracy: 0.5033 - loss: 2.0090
157/157 ━━━━━━━━━━━━━━━━━━━━ 29s 185ms/step


In [41]:
test_reviews = dp_test['text'].apply(pre_process_data)
test_reviews

0        got 'new' tires within two weeks got flat. too...
1        waste time. two different people come house gi...
2        say worst! people place lunch, place freezing ...
3        restaurant twice disappointed times. go back. ...
4        Food GOOD all! husband & ate couple weeks ago ...
                               ...                        
49995    wanted write review chip others said. would re...
49996    Great ambience. Great drinks. Great food. love...
49997    Monks locations excited heard coming Sun Prair...
49998    go here. know might want try good reviews peop...
49999    Buffet recently open renovation husband thinki...
Name: text, Length: 50000, dtype: object

In [47]:
tokenizer = Tokenizer(10000)
tokenizer.fit_on_texts(test_reviews)
sequences = tokenizer.texts_to_sequences(test_reviews)
max_length = np.max(list(map(lambda x: len(x), sequences)))
#print(max_length)
#pad all the input's to be the same length of the max length
test_inputs = pad_sequences(sequences, maxlen=755, padding = 'post')
test_inputs

array([[  17, 2079,  730, ...,    0,    0,    0],
       [ 799,    8,   45, ...,    0,    0,    0],
       [  62,  335,   27, ...,    0,    0,    0],
       ...,
       [1173,  627,  489, ...,    0,    0,    0],
       [  12,   66,   40, ...,    0,    0,    0],
       [ 244,  753,  227, ...,    0,    0,    0]], dtype=int32)

In [48]:
y_test = np.asarray(dp_test['label'])
y_test = tf.one_hot(y_test, depth=5)
y_test = tf.cast(y_test, dtype=tf.int32)
y_test

<tf.Tensor: shape=(50000, 5), dtype=int32, numpy=
array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0]], dtype=int32)>

In [49]:
model.evaluate(test_inputs, y_test)
#model.predict(test_inputs, y_test)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.2462 - loss: 8.0397 - precision_2: 0.2475 - recall_2: 0.2270


[8.197442054748535,
 0.24532000720500946,
 0.24672579765319824,
 0.22643999755382538]